<a href="https://colab.research.google.com/github/BIM-Master/Hotel-Reviews/blob/main/Hotel_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install pandas

In [8]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [2]:
pip install nltk

In [3]:
import nltk

In [4]:
from nltk.tokenize import word_tokenize

In [5]:
import pandas as pd

In [6]:
Hotel_df = pd.read_csv("/content/Hotel_Reviews 2.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/Hotel_Reviews 2.csv'

In [ ]:
Hotel_df

Removing attributes

In [11]:
columns_to_remove = ["Hotel_Address", "Additional_Number_of_Scoring", "Days_since_review", "lat", "lng"]

Hotel_df = Hotel_df.drop(columns=columns_to_remove, errors='ignore')

"Naming, Hotel_df cleaned & test it"

Tokenization

In [ ]:
nltk.download('punkt')

In [ ]:
Hotel_df['Negative_Review_tokenized'] = Hotel_df['Negative_Review'].apply(lambda x: word_tokenize(x))
Hotel_df['Positive_Review_tokenized'] = Hotel_df['Positive_Review'].apply(lambda x: word_tokenize(x))


In [ ]:
Hotel_df

Stop word removal

In [14]:
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
def remove_stop_words(tokens):
    stop_words = set(stopwords.words('english'))
    stop_words.discard('no')
    stop_words.discard('not')
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens

Test if "No" and "Not" are not removed during stop words removal

In [ ]:
Hotel_df['Negative_Review_tokenized_SWR'] = Hotel_df['Negative_Review_tokenized'].apply(remove_stop_words)
Hotel_df['Positive_Review_tokenized_SWR'] = Hotel_df['Positive_Review_tokenized'].apply(remove_stop_words)

In [ ]:
Hotel_df

POS-tagging, might be redundant based on how the code looks after lemmitization

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
from nltk import pos_tag

In [ ]:
Hotel_df['Negative_Review_tokenized_SWR_POS'] = Hotel_df['Negative_Review_tokenized_SWR'].apply(lambda x: pos_tag(x))
Hotel_df['Positive_Review_tokenized_SWR_POS'] = Hotel_df['Positive_Review_tokenized_SWR'].apply(lambda x: pos_tag(x))

In [ ]:
Hotel_df

Lemmatization part

In [22]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
from nltk.stem import WordNetLemmatizer

In [24]:
lemmatizer = WordNetLemmatizer()

In [25]:
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

In [26]:
def lemmatize_tokens(tagged_tokens):
    lemmatized_tokens = []
    for token, pos_tag in tagged_tokens:
        pos = pos_tag[0].lower() if pos_tag else 'n'
        pos = pos if pos in ['a', 'r', 'n', 'v'] else 'n'
        lemmatized_tokens.append(lemmatizer.lemmatize(token, pos))
    return lemmatized_tokens

In [27]:
Hotel_df['Negative_Review_tokenized_SWR_POS_Lem'] = Hotel_df['Negative_Review_tokenized_SWR_POS'].apply(lemmatize_tokens)
Hotel_df['Positive_Review_tokenized_SWR_POS_Lem'] = Hotel_df['Positive_Review_tokenized_SWR_POS'].apply(lemmatize_tokens)

In [30]:
Hotel_df

,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,...,Tags,days_since_review,Negative_Review_tokenized,Positive_Review_tokenized,Negative_Review_tokenized_SWR,Positive_Review_tokenized_SWR,Negative_Review_tokenized_SWR_POS,Positive_Review_tokenized_SWR_POS,Negative_Review_tokenized_SWR_POS_Lem,Positive_Review_tokenized_SWR_POS_Lem
0,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,...,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,"[I, am, so, angry, that, i, made, this, post, ...","[Only, the, park, outside, of, the, hotel, was...","[angry, made, post, available, via, possible, ...","[park, outside, hotel, beautiful]","[(angry, JJ), (made, VBD), (post, NN), (availa...","[(park, NN), (outside, JJ), (hotel, NN), (beau...","[angry, make, post, available, via, possible, ...","[park, outside, hotel, beautiful]"
1,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,...,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,"[No, Negative]","[No, real, complaints, the, hotel, was, great,...","[No, Negative]","[No, real, complaints, hotel, great, great, lo...","[(No, DT), (Negative, NNP)]","[(No, DT), (real, JJ), (complaints, NNS), (hot...","[No, Negative]","[No, real, complaint, hotel, great, great, loc..."
2,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,...,"[' Leisure trip ', ' Family with young childre...",3 days,"[Rooms, are, nice, but, for, elderly, a, bit, ...","[Location, was, good, and, staff, were, ok, It...","[Rooms, nice, elderly, bit, difficult, rooms, ...","[Location, good, staff, ok, cute, hotel, break...","[(Rooms, NNP), (nice, RB), (elderly, RB), (bit...","[(Location, NN), (good, JJ), (staff, NN), (ok,...","[Rooms, nice, elderly, bit, difficult, room, t...","[Location, good, staff, ok, cute, hotel, break..."
3,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,...,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,"[My, room, was, dirty, and, I, was, afraid, to...","[Great, location, in, nice, surroundings, the,...","[room, dirty, afraid, walk, barefoot, floor, l...","[Great, location, nice, surroundings, bar, res...","[(room, NN), (dirty, NN), (afraid, JJ), (walk,...","[(Great, NNP), (location, NN), (nice, JJ), (su...","[room, dirty, afraid, walk, barefoot, floor, l...","[Great, location, nice, surroundings, bar, res..."
4,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,...,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,"[You, When, I, booked, with, your, company, on...","[Amazing, location, and, building, Romantic, s...","[booked, company, line, showed, pictures, room...","[Amazing, location, building, Romantic, setting]","[(booked, VBN), (company, NN), (line, NN), (sh...","[(Amazing, VBG), (location, NN), (building, NN...","[book, company, line, show, picture, room, thi...","[Amazing, location, building, Romantic, setting]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,location,2,8,...,"[' Leisure trip ', ' Family with older childre...",704 day,"[no, trolly, or, staff, to, help, you, take, t...",[location],"[no, trolly, staff, help, take, luggage, room]",[location],"[(no, DT), (trolly, RB), (staff, NN), (help, N...","[(location, NN)]","[no, trolly, staff, help, take, luggage, room]",[location]
515734,8/22/2015,8.1,A

Feature Extraction, TD-IDF

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the text data
Negative_Review_tokenized_SWR_POS_Lem_tfidf = vectorizer.fit_transform(Hotel_df['Negative_Review_tokenized_SWR_POS_Lem'].apply(' '.join))
Positive_Review_tokenized_SWR_POS_Lem_tfidf = vectorizer.fit_transform(Hotel_df['Positive_Review_tokenized_SWR_POS_Lem'].apply(' '.join))




In [ ]:
# Convert to DataFrame
Negative_Review_tokenized_SWR_POS_Lem_tfidf_df = pd.DataFrame(Negative_Review_tokenized_SWR_POS_Lem_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
Positive_Review_tokenized_SWR_POS_Lem_tfidf_df = pd.DataFrame(Positive_Review_tokenized_SWR_POS_Lem_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

# Concatenate the TF-IDF features with the original DataFrame
Hot_df = pd.DataFrame()
Hot_df = pd.concat([df, Negative_Review_tokenized_SWR_POS_Lem_tfidf_df, Positive_Review_tokenized_SWR_POS_Lem_tfidf_df], axis=1)

Sentiment Analysis